# Exercises

https://swcarpentry.github.io/sql-novice-survey/10-prog/index.html

#### 1. Filling a Table vs. Printing Values

Write a Python program that creates a new database in a file called original.db containing a single table called Pressure, with a single field called reading, and inserts 100,000 random numbers between 10.0 and 25.0. How long does it take this program to run? How long does it take to run a program that simply writes those random numbers to a file?

In [11]:
import sqlite3
from numpy.random import uniform

# Create a list of 100,000 random numbers between 10.0 and 25.0
# answer key: list_of_numbers = uniform(low=10.0, high=25.0, size=100000)
list_of_numbers = []
for i in range(0, 100000):
    num = uniform(10.0, 25.0)
    list_of_numbers.append(num) 

connection = sqlite3.connect("original.db")
cursor = connection.cursor()
# Insert values into the students table
cursor.execute('''CREATE TABLE IF NOT EXISTS Pressure (reading INTEGER);''')
# Loop over the list of random numbers
for num in list_of_numbers:
    cursor.execute('''INSERT INTO Pressure VALUES (?)''', [num])

# Close the cursor
cursor.close()
# Commit changes to database (save changes for the next exercise)
connection.commit()
connection.close()

In [12]:
# In comparison, the following program writes the random numbers into
# the file random_numbers.txt
from numpy.random import uniform

random_numbers = uniform(low=10.0, high=25.0, size=100000)

with open('random_numbers.txt', 'w') as outfile:
    for number in random_numbers:
        outfile.write("{}\n".format(number))

In [14]:
# Read the file that was just written
with open('random_numbers.txt') as readfile:
    read_data = readfile.read()
print(read_data)

22.56832548870863
14.50754011161014
21.553831949526817
15.019089845538842
11.748644992609858
13.111286975780686
12.38367581718926
14.533632642644136
22.03541909060457
11.654022014327866
21.951345103446084
12.286477639159624
13.262615165033178
11.626824390398323
12.458190077482078
12.508368902735354
23.3476103967527
22.49069181306355
14.579140824488562
20.201349919421013
20.088758976667986
23.6311061058299
20.862775749358615
15.846040611738466
14.425701891193437
23.011954609565933
22.46213795109969
21.159132857042763
24.589569343182788
23.128441664389946
16.818527245482365
10.61841052131134
18.14432226952134
23.138783000192642
13.957288685840123
14.946335442371605
13.050950451003857
24.50889680727996
10.560017570447792
16.74541894581019
23.19736188311372
22.40007843541325
15.57288106161092
13.466357588560872
17.767443350001628
22.335012579128982
21.53770590690572
10.013658657451968
20.55309198788227
11.459757492314228
20.460789568431174
22.658577135042428
13.004361551247412
10.753163840

#### 2. Filtering in SQL vs. Filtering in Python
Write a Python program that creates a new database called backup.db with the same structure as original.db and copies all the values greater than 20.0 from original.db to backup.db. Which is faster: filtering values in the query, or reading everything into memory and filtering in Python?

In [20]:
# METHOD 1: FILTERING VALUES IN THE QUERY
# Get the values greater than 20.0
import sqlite3

connection_original = sqlite3.connect("original.db")
cursor_original = connection_original.cursor()

query = "SELECT * FROM Pressure WHERE reading > 20.0 "
cursor_original.execute(query)
numbers_over_20 = cursor_original.fetchall()
cursor_original.close()
connection_original.close()

#print(numbers_over_20)

connection_backup = sqlite3.connect("backup.db")
cursor_backup = connection_backup.cursor()
cursor_backup.execute("CREATE TABLE IF NOT EXISTS Pressure (reading float not null)")
query_2 = "INSERT INTO Pressure (reading) VALUES (?);"

for num in numbers_over_20:
    cursor_backup.execute(query_2, [num[0]])
cursor_backup.close()
connection_backup.commit()
connection_backup.close()

[(22.669174609340224,), (20.169455777045528,), (21.584638767718005,), (22.299384513682753,), (21.79860417882812,), (21.726514933845912,), (22.72701992257975,), (24.55449481426624,), (21.226858967481405,), (21.211426648217927,), (21.226853732806738,), (20.205976295045318,), (24.775166624974773,), (23.618953441858494,), (21.503746190175526,), (22.00302360435372,), (22.95721318653225,), (20.089773309599185,), (22.148721720488282,), (22.787280305086295,), (20.183522375005072,), (22.9169990179564,), (22.262118710970356,), (23.566662393089516,), (21.917616949537027,), (23.098879450402592,), (21.007880060603988,), (20.700956692465873,), (24.254192029325967,), (24.28015143493407,), (21.162499117750045,), (24.336401550713806,), (20.651936371386853,), (22.72826076315331,), (20.239440818274097,), (23.320244276228685,), (23.55908563064611,), (24.27945741956978,), (20.59669371145683,), (21.691657039780182,), (23.70571847677705,), (21.835427740021963,), (22.828605052894858,), (24.381053046787134,), 

In [22]:
# METHOD 2: READING EVERYTHING INTO MEMORY AND FILTERING IN PYTHON
import sqlite3

connection_original = sqlite3.connect("original.db")
cursor_original = connection_original.cursor()

query = "SELECT * FROM Pressure;"
cursor_original.execute(query)
numbers_over_20 = cursor_original.fetchall()
cursor_original.close()
connection_original.close()

#print(numbers_over_20)

connection_backup = sqlite3.connect("backup.db")
cursor_backup = connection_backup.cursor()
cursor_backup.execute("CREATE TABLE IF NOT EXISTS Pressure (reading float not null)")
query_2 = "INSERT INTO Pressure (reading) VALUES (?);"

for num in numbers_over_20:
    if num[0] > 20.0:
        cursor_backup.execute(query_2, num)
cursor_backup.close()
connection_backup.commit()
connection_backup.close()

#### 3. Generating Insert Statements

One of our colleagues has sent us a CSV file containing temperature readings by Robert Olmstead, which is formatted like this:

Taken,Temp
619,-21.5
622,-15.5

Write a small Python program that reads this file in and prints out the SQL INSERT statements needed to add these records to the survey database. Note: you will need to add an entry for Olmstead to the Person table. If you are testing your program repeatedly, you may want to investigate SQL’s INSERT or REPLACE command.

In [48]:
import csv
def gen_insert_statements(read_csv):
    with open(read_csv, encoding='utf-8-sig') as read_f:
        read_data = csv.DictReader(read_f)
        print(read_data.fieldnames)
        # Keys: 'Taken', 'Temp'  
        for row in read_data:
            query = ''' INSERT INTO table_name (Taken, Temp) VALUES ({value1}, {value2})
            '''.format(value1=row["Taken"], value2=row["Temp"])
            print(query)

In [50]:
gen_insert_statements("temp_readings.csv")

['Taken', 'Temp']
 INSERT INTO table_name (Taken, Temp) VALUES (619, -21.5)
            
 INSERT INTO table_name (Taken, Temp) VALUES (622, -15.5)
            
